# KL-Sum: Implementation and Evaluation

Notebook implements the KL-Sum extractive summarization algorithm, evaluates it with ROUGE, BERTScore, and Cosine Similarity, and provides example experiments.

**Contents:**
1. Install & imports
2. Data loading example
3. KL-Sum implementation (greedy selection minimizing KL divergence)
4. Evaluation: ROUGE, BERTScore, Cosine Similarity (with sentence embeddings)
5. Example run on a small dataset


In [1]:
# 1) Install required packages (the cell will try to pip-install missing packages)
import sys
import subprocess

def pip_install(pkg):
    try:
        __import__(pkg)
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

# Map pip names to import names where necessary
packages = [
    ("nltk", "nltk"),
    ("numpy", "numpy"),
    ("pandas", "pandas"),
    ("scikit-learn", "sklearn"),
    ("rouge-score", "rouge_score"),
    ("bert-score", "bert_score"),
    ("sentence-transformers", "sentence_transformers"),
    ("tqdm", "tqdm"),
]

for pip_name, import_name in packages:
    try:
        __import__(import_name)
    except Exception:
        print(f"Installing {pip_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])

# Imports
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import pandas as pd
import math
from collections import Counter, defaultdict
from math import log

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
from bert_score import score as bertscore_score
from sentence_transformers import SentenceTransformer

print('Imports done.')

c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imports done.


In [2]:
# 2) KL-Sum implementation (greedy extractive summarizer)
from copy import deepcopy
from collections import Counter
import re

def unigram_distribution(text_tokens, vocab=None, alpha=1.0):
    '''Return a dict of unigram probabilities with Laplace smoothing (alpha).'''
    cnt = Counter(text_tokens)
    if vocab is None:
        vocab = set(cnt.keys())
    V = len(vocab)
    total = sum(cnt.values())
    probs = {}
    for w in vocab:
        probs[w] = (cnt.get(w,0) + alpha) / (total + alpha * V)
    return probs

def kl_divergence(p, q):
    '''Compute D_KL(p || q) for discrete distributions given as dicts.Assumes support of p is subset of q; uses small eps if q=0.'''
    eps = 1e-12
    s = 0.0
    for w, pw in p.items():
        qw = q.get(w, eps)
        s += pw * math.log(pw / qw)
    return s

def sentence_split_vi(text):
    '''A simple sentence splitter for Vietnamese using punctuation.'''
    # This is a naive implementation; for better results, consider using a dedicated Vietnamese NLP library.
    sentences = re.split(r'(?<=[.!?]) +', text)
    return [s.strip() for s in sentences if s.strip()]



def kl_sum_summarize(document, max_sentences=3, smoothing_alpha=1.0):
    '''Greedy KL-Sum: build summary by adding sentences that most reduce D(P||Q_S).'''
    sentences = sentence_split_vi(document)
    # tokenize words (simple whitespace + nltk word_tokenize)
    sent_tokens = [ [w.lower() for w in word_tokenize(s)] for s in sentences ]
    # overall document unigram distribution
    all_tokens = [w for toks in sent_tokens for w in toks]
    vocab = set(all_tokens)
    P = unigram_distribution(all_tokens, vocab=vocab, alpha=smoothing_alpha)
    # initialize summary as empty
    selected = []
    selected_tokens = []
    remaining = set(range(len(sentences)))
    # Greedy loop
    for _ in range(min(max_sentences, len(sentences))):
        best_choice = None
        best_kl = None
        for i in remaining:
            candidate_tokens = selected_tokens + sent_tokens[i]
            Q = unigram_distribution(candidate_tokens, vocab=vocab, alpha=smoothing_alpha)
            kl = kl_divergence(P, Q)
            if best_kl is None or kl < best_kl:
                best_kl = kl
                best_choice = i
        if best_choice is None:
            break
        # add chosen sentence
        selected.append(best_choice)
        selected_tokens += sent_tokens[best_choice]
        remaining.remove(best_choice)
    # return sentences in original order
    selected_sorted = sorted(selected)
    summary = ' '.join([sentences[i] for i in selected_sorted])
    return summary

# Quick sanity test
doc = "Natural language processing (NLP) is a field of artificial intelligence. It focuses on the interaction between computers and human language. Summarization reduces text to its essential ideas."
print('KL-Sum summary:', kl_sum_summarize(doc, max_sentences=3))

KL-Sum summary: Natural language processing (NLP) is a field of artificial intelligence. It focuses on the interaction between computers and human language. Summarization reduces text to its essential ideas.


In [3]:
# 3) Evaluation utilities: ROUGE, BERTScore, Cosine similarity (embeddings or tf-idf)

# ROUGE scorer
rouge = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

def compute_rouge(reference, hypothesis):
    scores = rouge.score(reference, hypothesis)
    # Return F1 scores
    return {k: v.fmeasure for k,v in scores.items()}

# BERTScore wrapper
def compute_bertscore_batch(refs, hyps, lang='en', model_type='bert-base-uncased'):
    # bert_score.score returns P, R, F: we will return F1 array and average
    P, R, F1 = bertscore_score(hyps, refs, lang=lang, rescale_with_baseline=True, model_type=model_type)
    return {'precision': P.tolist(), 'recall': R.tolist(), 'f1': F1.tolist(), 'f1_avg': float(F1.mean())}

# Cosine similarity using SentenceTransformer embeddings
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

def compute_cosine_embedding(ref, hyp):
    emb = embed_model.encode([ref, hyp])
    cos = cosine_similarity(emb[0:1], emb[1:2])[0,0]
    return float(cos)

In [47]:
# 4) Main loop: load data, summarize, evaluate, print results
from tqdm import tqdm

df = pd.read_csv("data.csv", encoding="utf-8")

df.rename(columns={"Document": "text", "Summary": "reference"}, inplace=True)

print(f" Loaded {len(df)} samples from data.csv\n")

# Lists to store scores
rouge1_list, rouge2_list, rougeL_list, bert_list, cosine_list = [], [], [], [], []

for i, row in tqdm(df.iterrows(), total=len(df), desc="Summarizing..."):
    doc = str(row['text'])
    ref = str(row['reference'])
    
    hyp = kl_sum_summarize(doc, max_sentences=2, smoothing_alpha=1.0)
    
    # Đánh giá
    rouge_score = compute_rouge(ref, hyp)
    bert_f1 = compute_bertscore_batch([ref], [hyp])['f1_avg']
    cosine_sim = compute_cosine_embedding(ref, hyp)
    
    # Lưu kết quả vào list
    rouge1_list.append(rouge_score['rouge1'])
    rouge2_list.append(rouge_score['rouge2'])
    rougeL_list.append(rouge_score['rougeL'])
    bert_list.append(bert_f1)
    cosine_list.append(cosine_sim)
    
    
    if i < 3:
        print(f"\n🔹 Sample {i+1}")
        print(f"DOC: {doc:10}...")
        print(f"HYP: {hyp}")
        print(f"REF: {ref}")
        print(f"ROUGE: {rouge_score}")
        print(f"BERT F1 avg: {bert_f1}")
        print(f"Cosine Similarity: {cosine_sim}")
        print('-'*100)

# Tính trung bình và in kết quả
print("\n======  TRUNG BÌNH TOÀN BỘ TẬP DỮ LIỆU ======")
print(f"ROUGE-1: {np.mean(rouge1_list):.4f}")
print(f"ROUGE-2: {np.mean(rouge2_list):.4f}")
print(f"ROUGE-L: {np.mean(rougeL_list):.4f}")
print(f"BERTScore F1: {np.mean(bert_list):.4f}")
print(f"Cosine Similarity: {np.mean(cosine_list):.4f}")
print("===============================================")


 Loaded 74564 samples from data.csv



Summarizing...:   0%|          | 1/74564 [00:03<80:07:36,  3.87s/it]


🔹 Sample 1
DOC: Đây là một trong những nội dung tại văn bản vừa được UBND TP Hà Nội ban hành về việc tăng cường công tác quản lý nuôi , giết mổ , kinh doanh và sử dụng thịt chó , mèo trên địa bàn .Theo đó , các sở , ngành trên địa bàn phải vào cuộc ngay để hướng tới thay đổi thói quen của người dân khi dùng chó , mèo làm thực phẩm .Gây phản cảm với du khách , người nước ngoàiCũng trong văn bản này , UBND TP Hà Nội thừa nhận rằng việc kinh doanh , giết mổ và sử dụng thịt chó , mèo tại Hà Nội thời gian qua đã tạo ra những hình ảnh phản cảm đối với khách tham quan du lịch , người nước ngoài đang sinh sống và làm việc tại Hà Nội , gây ảnh hưởng tới hình ảnh của một thủ đô " văn minh , hiện đại " .Trong thực tế , theo ghi nhận của Tuổi Trẻ tại phố Tam Trinh , ngay đoạn đầu cầu Mai Động ( quận Hoàng Mai ) , một đoạn phố dài với gần chục cửa hàng buôn bán thịt chó sống , nhà hàng phục vụ các món chế biến từ thịt chó vẫn hoạt động tấp nập nhiều năm nay .Chị Th . , một chủ sạp bán thịt chó sốn

Summarizing...:   0%|          | 2/74564 [00:06<68:27:51,  3.31s/it]


🔹 Sample 2
DOC: Chiều 12/3 , ông Vũ Hùng Triều , Trưởng phòng Tổ chức hành chính Bệnh viện Đa khoa huyện Đông Hoà ( Phú Yên ) cho biết , bé trai sơ sinh bị bỏ rơi được người dân đưa đến cấp cứu sức khoẻ đã dần ổn định ." Bệnh viện sẽ tiếp tục theo dõi , điều trị chống nhiễm trùng cho bé và chờ chính quyền làm thủ tục xác định sự việc để làm giấy chứng sinh cho người bảo hộ " , ông này nói .Bé trai mới sinh nặng 2,7 kg bị bỏ rơi hiện được chăm sóc ở bệnh viện .Trước đó rạng sáng 11/3 , Bệnh viện Đông Hoà tiếp nhận cháu bé từ tay chị Nguyễn Thị Tâm ( 32 tuổi , xã Hoà Hiệp Nam , huyện Đông Hoà ) .Theo chị Tâm , 5h sáng , sau khi ngủ dậy chị mở cửa ra sân thì phát hiện chiếc thùng carton đặt gần mép sân , bên trong có bé trai sơ sinh chưa cắt rốn , cơ thể tím tái , thở yếu , trên người có nhiều vết kiến đốt .Chị Tâm đã hô hoán nhờ nhiều người đến trợ giúp , sau đó gọi y tá đến cắt rốn và chở đến bệnh viện cấp cứu .Nhiều người đến Bệnh viện Đông Hoà chăm bé trai khi chị Tâm không có mặt .C

Summarizing...:   0%|          | 3/74564 [00:09<63:31:31,  3.07s/it]


🔹 Sample 3
DOC: Thoạt đầu nhìn vào bức ảnh , nếu không có dấu khoanh tròn , rất khó nhận ra chú báo tuyết .Bộ lông của báo tuyết giúp chúng ẩn thân trong môi trường sống đặc thù , hình thức nguỵ trang này khiến chúng rất khó bị phát hiện khi săn mồi .Matthias Fiechter , thành viên tổ chức Snow Leopard Trust cho biết bộ lông của loài báo tuyết chính là vũ khí hiệu quả giúp chúng khi săn mồi ." Nhờ có bộ lông và màu sắc độc đáo , báo tuyết có thể ẩn mình hoàn hảo với môi trường xung quanh , sau đó tấn công con mồi một cách chớp nhoáng " .Báo tuyết khá nhút nhát và thích cuộc sống đơn độc , thường hoạt động tích cực nhất vào lúc bình minh và hoàng hôn để tuần tra lãnh thổ và săn mồi .Báo tuyết ưa thích sinh sống tại những nơi có địa hình hiểm trở , lạnh lẽo và cằn cỗi trên các dãy núi đá cao trên 1.000 m so với mực nước biển .Trên dãy Himalaya , loài này thường được tìm thấy tại độ cao từ 3.000-5.400 m so với mực nước biển .Phạm vi sinh sống của loài này trải dài trên lãnh thổ 12 quốc gi

Summarizing...:  62%|██████▏   | 46339/74564 [23:31:50<53:53:26,  6.87s/it]'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f623d185-90da-45cf-8405-c3c158f0e7cc)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
Summarizing...:  62%|██████▏   | 46372/74564 [23:34:29<64:30:27,  8.24s/it]'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 642b8c7c-be92-4ef6-a631-94f2589ce70a)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tokenizer_config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001ED11E410


======  TRUNG BÌNH TOÀN BỘ TẬP DỮ LIỆU ======
ROUGE-1: 0.3953
ROUGE-2: 0.1795
ROUGE-L: 0.2510
BERTScore F1: 0.5654
Cosine Similarity: 0.6960


In [10]:
# 4) Main loop: load data, summarize, evaluate, print results

df = pd.read_csv("test.csv", encoding="utf-8")

print(f"Loaded {len(df)} samples from test.csv\n")

# Tạo list lưu kết quả
rouge1_list = []
rouge2_list = []
rougeL_list = []
bert_list = []
cosine_list = []

for i, row in tqdm(df.iterrows(), total=len(df), desc="Summarizing..."):
    doc = str(row['text'])
    ref = str(row['summary'])
    
    # Tạo tóm tắt bằng KL-Sum
    hyp = kl_sum_summarize(doc, max_sentences=2, smoothing_alpha=1.0)
    
    # Đánh giá
    rouge_score = compute_rouge(ref, hyp)
    bert_f1 = compute_bertscore_batch([ref], [hyp])['f1_avg']
    cosine_sim = compute_cosine_embedding(ref, hyp)
    
    # Lưu vào list
    rouge1_list.append(rouge_score['rouge1'])
    rouge2_list.append(rouge_score['rouge2'])
    rougeL_list.append(rouge_score['rougeL'])
    bert_list.append(bert_f1)
    cosine_list.append(cosine_sim)
    
    # In từng mẫu
    print(f"\n🔹 Sample {i+1}")
    print(f"DOC: {doc[:70]}...")
    print(f"HYP: {hyp}")
    print(f"REF: {ref}")
    print(f"ROUGE: {rouge_score}")
    print(f"BERT F1 avg: {bert_f1}")
    print(f"Cosine Similarity: {cosine_sim}")
    print('-'*100)

# Tính trung bình
print("\n======  TRUNG BÌNH TOÀN BỘ TẬP DỮ LIỆU ======")
print(f"ROUGE-1: {np.mean(rouge1_list):.4f}")
print(f"ROUGE-2: {np.mean(rouge2_list):.4f}")
print(f"ROUGE-L: {np.mean(rougeL_list):.4f}")
print(f"BERTScore F1: {np.mean(bert_list):.4f}")
print(f"Cosine Similarity: {np.mean(cosine_list):.4f}")
print("===============================================")


Loaded 178 samples from test.csv



Summarizing...:   1%|          | 1/178 [00:02<07:40,  2.60s/it]


🔹 Sample 1
DOC: 
Sau bài thi ngắn năm mươi phút và bốn mươi câu hỏi trắc nghiệm, phần ...
HYP: Đại thụ của nền sử học ra đi, để lại cho cuộc đời một câu hỏi: bao giờ những khoảng trống lịch sử được lấp đầy?
Trong bốn mươi câu trắc nghiệm lịch sử sáng nay, sẽ còn thiếu rất nhiều sự kiện và sự thật lịch sử quan trọng của Việt Nam, mà ngành giáo dục vẫn còn chưa thống nhất được với ngành lịch sử, với những nhà nghiên cứu trăn trở như Phan Huy Lê, rằng chúng có nên được đưa vào và sẽ được đưa vào sách giáo khoa như thế nào.
Lần đầu tiên tôi nhận ra sự thiếu hụt trong kiến thức lịch sử của mình, là khi đã trở thành sinh viên năm thứ ba của trường báo. Đó là lần đầu trong đời, tôi được mắt thấy tai nghe các sử gia hàng đầu đất nước công khai những những khoảng trống lịch sử lâu nay bị coi là “nhạy cảm”.
Trong buổi hội thảo ấy, giáo sư Lê đề nghị xác lập quan điểm mới khi làm sử là phải toàn bộ, toàn diện.
REF: Đại thụ của nền sử học ra đi, để lại cho cuộc đời một câu hỏi: bao giờ những khoả

Summarizing...:   1%|          | 2/178 [00:04<06:44,  2.30s/it]


🔹 Sample 2
DOC: Với doanh thu bình quân 200 USD/TEU, theo số liệu của một tập đoàn kha...
HYP: Thách thức của Cần Giờ nằm ở chỗ làm sao có thể chiếm được dòng tàu từ các siêu cảng này, đặc biệt Singapore và Tanjung Pelepas, vốn sở hữu công nghệ khai thác hiện đại, mạng lưới kết nối đường biển dày đặc đã hình thành trong thời gian dài, và đặc biệt là vị trí chiến lược bậc nhất của eo biển Malacca trong hệ thống vận tải toàn cầu. Không chỉ là đầu mối cho Đông Nam Á trên trục Đông - Tây (Châu Á - Châu Âu - Bắc Mỹ), các cảng này còn là trạm trung chuyển hàng hóa với Trung Đông và Nam Á, châu Đại Đương và châu Phi.Theo khảo sát gần đây của chúng tôi đối với mạng lưới vận tải container quốc tế, ba cảng này chiếm 68% lượt tàu hàng tuần của MSC (số một thế giới), và 51% của CMA-CGM (số ba) tại Đông Nam Á.
REF: Với doanh thu bình quân 200 USD/TEU, theo số liệu của một tập đoàn khai thác cảng, hàng năm cảng có thể thu về 2-3 tỷ USD, chưa kể doanh thu từ các hoạt động phụ trợ.Cửa ngõ (gateway) v

...


🔹 Sample 176
DOC: Người này ngày nào cũng gọi video hỏi xem hôm nay tôi thấy trong người...
HYP: Người này ngày nào cũng gọi video hỏi xem hôm nay tôi thấy trong người thế nào, có đau họng, ho hay sốt gì không, thân nhiệt cao hay thấp, rồi còn bắt lia camera quanh phòng xem tôi ở một mình hay cùng ai.Đó là nhân viên của MOM - Bộ Nhân lực Singapore, gọi video qua Whatsapp hàng ngày để theo dõi sức khoẻ và giám sát việc tuân thủ cách ly.Chuyến bay đầu tiên của tôi sau mùa dịch là từ Hà Nội sang Singapore. Tôi vẫn phải thuê một phòng nằm trong danh sách khách sạn được phép SHN của nước này.Xuống sân bay Changi, nhân viên hướng dẫn tôi đến nơi làm tờ khai, đo thân nhiệt, kiểm tra giấy chứng nhận tiêm vaccine cùng xét nghiệm PCR âm tính ở Việt Nam.
REF: Người này ngày nào cũng gọi video hỏi xem hôm nay tôi thấy trong người thế nào, có đau họng, ho hay sốt gì không, thân nhiệt cao hay thấp, rồi còn bắt lia camera quanh phòng xem tôi ở một mình hay cùng ai.Đó là nhân viên của MOM - Bộ Nhân l

Summarizing...:  99%|█████████▉| 177/178 [06:34<00:02,  2.09s/it]


🔹 Sample 177
DOC: Công việc bàn giấy, cộng với thói quen ăn uống, sinh hoạt thiếu điều đ...
HYP: Lối sống thiếu vận động, ngồi nhiều, ăn uống thiếu điều độ, thức khuya, sử dụng chất kích thích là những nguyên nhân trực tiếp dẫn tới các căn bệnh không lây nhiễm như tim mạch, đái tháo đường, ung thư, loãng xương, gout...Quỹ Dân số Liên Hợp Quốc (UNFPA) ước tính, hàng năm, Việt Nam có khoảng 350.000 người chết vì các bệnh không lây nhiễm. Kinh nghiệm ở nhiều quốc gia cho thấy nếu nhà nước có chính sách chăm lo cho các hoạt động thể thao trong cộng đồng thì sẽ thu được lợi ích về nhiều mặt như cải thiện chất lượng nguồn nhân lực, giảm tệ nạn xã hội, giảm gánh nặng y tế...Tôi hy vọng một ngày nào đó, tỷ lệ sân tập, điểm tập thể thao cộng đồng tính trên số dân ở Việt Nam sẽ cao hơn tỷ lệ nhà thuốc trên số dân.
REF: Thống kê cho thấy một đơn trung bình có bốn loại thuốc thì hai trong số đó không phải là thuốc thiết yếu cần kê cho bệnh nhân.Nhưng dù giàu hay nghèo, người bệnh hầu như không có

Summarizing...: 100%|██████████| 178/178 [06:36<00:00,  2.23s/it]


🔹 Sample 178
DOC: 
Độc giả nào từng thất vọng về những giả dối đó sẽ bớt muộn phiền hơn ...
HYP: Những bùng phát bạo lực và hành vi quá khích của đám đông vừa qua theo tôi, có một phần hệ quả từ đó.
Để hạn chế những tác động tiêu cực của ý nguyện che giấu, đương nhiên, là khiến nó không còn bị “che giấu” nữa. Điều này, như bài viết của tôi đề xuất cách đây ba năm, bao gồm việc nâng cao chất lượng của các tổ chức trung gian - các đoàn thể quần chúng và tổ chức xã hội - đồng thời tạo cơ sở để người dân bày tỏ ý kiến trực tiếp bằng những khung pháp lý cụ thể.
Trên tất cả, tôi tin rằng không một ai muốn giữ mãi trong lòng những ấm ức của mình.
REF: Ngoài công sở, chúng ta có thể tặc lưỡi đóng góp một khoản từ thiện khi mọi người xung quanh đều làm, hay gật đầu đồng ý với tất cả các câu hỏi từ tổng đài chăm sóc khách hàng của một cửa hiệu mới mua đồ vì ngại mất thời gian.
Thực ra trong cuộc sống, không ai trong chúng ta có thể 100% nói ra hết những gì mình nghĩ: vì sợ người khác tổn thương

In [48]:
# 4) Example: run KL-Sum on sample documents
documents = [
    "NASA announced a new mission to study Mars. The mission will launch next year and will carry a rover to investigate the surface.",
    "The stock market rallied today after tech earnings beat expectations. Investors hope the momentum continues."
]
references = [
    "NASA announced a new Mars mission with a rover launching next year.",
    "Stock market rallied after strong tech earnings, boosting investor optimism."
]

for doc, ref in zip(documents, references):
    hyp = kl_sum_summarize(doc, max_sentences=1)
    print('DOC:', doc)
    print('HYP:', hyp)
    print('REF:', ref)
    print('ROUGE:', compute_rouge(ref, hyp))
    print('BERT F1 avg:', compute_bertscore_batch([ref],[hyp])['f1_avg'])
    print('Cosine sim:', compute_cosine_embedding(ref, hyp))
    print('-'*80)

DOC: NASA announced a new mission to study Mars. The mission will launch next year and will carry a rover to investigate the surface.
HYP: The mission will launch next year and will carry a rover to investigate the surface.
REF: NASA announced a new Mars mission with a rover launching next year.
ROUGE: {'rouge1': 0.4444444444444445, 'rouge2': 0.23999999999999996, 'rougeL': 0.2962962962962963}
BERT F1 avg: 0.5301455855369568
Cosine sim: 0.6850239038467407
--------------------------------------------------------------------------------
DOC: The stock market rallied today after tech earnings beat expectations. Investors hope the momentum continues.
HYP: The stock market rallied today after tech earnings beat expectations.
REF: Stock market rallied after strong tech earnings, boosting investor optimism.
ROUGE: {'rouge1': 0.6, 'rouge2': 0.3333333333333333, 'rougeL': 0.6}
BERT F1 avg: 0.6008275747299194
Cosine sim: 0.9339042901992798
----------------------------------------------------------

In [52]:
doc = "Người đầu tiên xứng với danh xưng hảo hán không ai khác chính la Võ Tòng. Cả đời Võ Tòng hành hiệp trượng nghĩa, thích bênh vực kẻ yếu, hoàn toàn giống với một người anh hùng. Hơn nữa Võ Tòng còn là một người yêu hận phân minh, không tàn sát người vô tội, đây là điểm rất quan trọng. Lấy chuyện Võ Tòng báo thù cho Võ Đại Lang làm ví dụ, Võ Tòng chỉ giết đúng hai kẻ liên quan trực tiếp là Phan Kim Liên và Tây Môn Khánh, Võ Tòng không hề ra tay với Vương bà dù bà ta cũng là người góp phần không nhỏ, đối với những người đi hóng chuyện, Võ Tòng cũng không ra tay thảo phạt hàng loạt, làm người rất có chừng mực, cho thấy bản chất đúng đắn của một anh hùng hảo hán.Đứng ở góc độ hiện tại, nhiều người có thể cảm thấy cách cư xử của Võ Tòng là điều hết sức bình thường, bởi lẽ chuyện này quả thực chẳng liên quan tới những người khác, nhưng đây lại chỉ là tư duy của người hiện đại chúng ta, trong Thủy Hử, không thiếu gì hảo hán chẳng cần biết vô tội hay không, chỉ cần động thủ là phải đánh cho tới bến, cho sảng khoái thì thôi. Nếu đổi lại là Lý Quỳ, có lẽ người bị giết sẽ nhiều hơn, chỉ cần là những người biết chuyện của Võ Đại Lang mà không chịu lên tiếng hay làm gì, e là có lẽ đều sẽ bị giết…Nhưng Võ Tòng thì khác, sau khi biết đầu đuôi câu chuyện, Võ Tòng chỉ oan có đầu nợ có chủ, chỉ nhắm vào đương sự, không lạm dụng vũ lực, đây mới chính là anh hùng hảo hán.Thủy Hử 108 anh hùng, chỉ 4 người thực sự là hảo hán gồm những aiNhân vật Võ Tòng trên màn ảnh nhỏ"
summary = kl_sum_summarize(doc, max_sentences=1)
ref="Võ Tòng xứng đáng là hình mẫu anh hùng hảo hán bởi tính cách trượng nghĩa, hành hiệp giúp người, yêu hận rõ ràng và không giết kẻ vô tội. Khi báo thù cho anh trai, Võ Tòng chỉ giết Phan Kim Liên và Tây Môn Khánh – những kẻ trực tiếp gây tội, thể hiện sự chừng mực và chính nghĩa, khác với nhiều hảo hán khác trong Thủy Hử thường hành động bạo liệt, thiếu kiềm chế."

print("\n Original doc:", doc)
print(" KL-Sum summary:", summary)
print(" Reference summary of chatgpt:", ref)

# Đánh giá nhanh
rouge_score = compute_rouge(ref, summary)
cos_sim = compute_cosine_embedding(ref, summary)
bert = compute_bertscore_batch([ref], [summary])

print("\n🔹 ROUGE:", rouge_score)
print("🔹 Cosine:", cos_sim)
print("🔹 BERTScore F1:", bert['f1_avg'])



 Original doc: Người đầu tiên xứng với danh xưng hảo hán không ai khác chính la Võ Tòng. Cả đời Võ Tòng hành hiệp trượng nghĩa, thích bênh vực kẻ yếu, hoàn toàn giống với một người anh hùng. Hơn nữa Võ Tòng còn là một người yêu hận phân minh, không tàn sát người vô tội, đây là điểm rất quan trọng. Lấy chuyện Võ Tòng báo thù cho Võ Đại Lang làm ví dụ, Võ Tòng chỉ giết đúng hai kẻ liên quan trực tiếp là Phan Kim Liên và Tây Môn Khánh, Võ Tòng không hề ra tay với Vương bà dù bà ta cũng là người góp phần không nhỏ, đối với những người đi hóng chuyện, Võ Tòng cũng không ra tay thảo phạt hàng loạt, làm người rất có chừng mực, cho thấy bản chất đúng đắn của một anh hùng hảo hán.Đứng ở góc độ hiện tại, nhiều người có thể cảm thấy cách cư xử của Võ Tòng là điều hết sức bình thường, bởi lẽ chuyện này quả thực chẳng liên quan tới những người khác, nhưng đây lại chỉ là tư duy của người hiện đại chúng ta, trong Thủy Hử, không thiếu gì hảo hán chẳng cần biết vô tội hay không, chỉ cần động thủ là ph

In [56]:
doc="Trong thế giới của Tam Quốc Diễn Nghĩa, Gia Cát Lượng được xem là biểu tượng cho trí tuệ và lòng trung thành. Từ khi ra khỏi lều cỏ Long Trung, ông đã gắn bó trọn đời với nhà Thục Hán, dốc hết tâm huyết giúp Lưu Bị dựng nghiệp, rồi sau đó phò tá Lưu Thiện cho đến lúc chết. Cả cuộc đời ông là minh chứng cho hai chữ “tận trung”, dù biết đại nghiệp khó thành vẫn không từ bỏ. Gia Cát Lượng không chỉ là một mưu sĩ tài ba mà còn là một nhà chính trị, một nhà quân sự toàn tài. Ông giỏi chiến lược, giỏi trị quốc, và đặc biệt luôn đặt lợi ích quốc gia lên trên danh vọng cá nhân. Điều khiến người đời ngưỡng mộ hơn cả là đức khiêm nhường và tinh thần cần cù của Gia Cát Lượng. Dù được người đời tôn xưng là “Ngọa Long”, ông chưa từng kiêu ngạo. Trước mỗi trận chiến, ông đều cân nhắc kỹ lưỡng, lấy mưu lược thay cho bạo lực, lấy nhân nghĩa để thu phục lòng người. Bức thư Giới tử thư mà ông để lại cho con trai, đến nay vẫn được xem là một áng văn dạy người tu thân, giữ đạo giữa đời nhiễu nhương. Tuy vậy, Gia Cát Lượng cũng không phải là người hoàn hảo. Sự quá cẩn trọng đôi khi khiến ông bỏ lỡ cơ hội, như trong chiến dịch Bắc phạt, dù có tài mưu lược nhưng lại thiếu người kế thừa, dẫn đến đại nghiệp Thục Hán dần suy yếu sau khi ông mất. Dẫu vậy, khi nhắc đến ông, người đời vẫn kính phục, bởi ông là hình ảnh của một bậc hiền thần – người biết rõ vận nước khó đổi nhưng vẫn chọn dốc hết sức mình để giữ lấy đạo trung quân ái quốc. Có thể nói, Gia Cát Lượng là tấm gương sáng về trí tuệ, nhân cách và tinh thần phụng sự mà hậu thế muôn đời noi theo."
summary = kl_sum_summarize(doc, max_sentences=2)
ref="Gia Cát Lượng là biểu tượng của trí tuệ và lòng trung thành trong Tam Quốc Diễn Nghĩa. Ông tận tâm giúp nhà Thục Hán, nổi tiếng với mưu lược và đức khiêm nhường, dù có những hạn chế như quá cẩn trọng trong chiến lược."
print("\n Original doc:", doc)
print(" KL-Sum summary:", summary)
print(" Reference summary of gemini:", ref)
# Đánh giá nhanh
rouge_score = compute_rouge(ref, summary)
cos_sim = compute_cosine_embedding(ref, summary)
bert = compute_bertscore_batch([ref], [summary])
print("\n🔹 ROUGE:", rouge_score)
print("🔹 Cosine:", cos_sim)
print("🔹 BERTScore F1:", bert['f1_avg'])


 Original doc: Trong thế giới của Tam Quốc Diễn Nghĩa, Gia Cát Lượng được xem là biểu tượng cho trí tuệ và lòng trung thành. Từ khi ra khỏi lều cỏ Long Trung, ông đã gắn bó trọn đời với nhà Thục Hán, dốc hết tâm huyết giúp Lưu Bị dựng nghiệp, rồi sau đó phò tá Lưu Thiện cho đến lúc chết. Cả cuộc đời ông là minh chứng cho hai chữ “tận trung”, dù biết đại nghiệp khó thành vẫn không từ bỏ. Gia Cát Lượng không chỉ là một mưu sĩ tài ba mà còn là một nhà chính trị, một nhà quân sự toàn tài. Ông giỏi chiến lược, giỏi trị quốc, và đặc biệt luôn đặt lợi ích quốc gia lên trên danh vọng cá nhân. Điều khiến người đời ngưỡng mộ hơn cả là đức khiêm nhường và tinh thần cần cù của Gia Cát Lượng. Dù được người đời tôn xưng là “Ngọa Long”, ông chưa từng kiêu ngạo. Trước mỗi trận chiến, ông đều cân nhắc kỹ lưỡng, lấy mưu lược thay cho bạo lực, lấy nhân nghĩa để thu phục lòng người. Bức thư Giới tử thư mà ông để lại cho con trai, đến nay vẫn được xem là một áng văn dạy người tu thân, giữ đạo giữa đời nhi